In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Set the video file path
VIDEO_PATH = r"C:\Users\nishi\store-analytics\yolov8\20250118_123057.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)

# Store person positions
heatmap_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO inference
    results = model(frame)

    # Extract person centroids and store them
    for result in results:
        for detection in result.boxes:
            if detection.cls == 0:  # Class 0 = 'person' in COCO dataset
                bbox = detection.xyxy[0].cpu().numpy().astype(int)
                centroid = ((bbox[0] + bbox[2]) // 2, (bbox[1] + bbox[3]) // 2)
                heatmap_data.append(centroid)

    # Display the video with bounding boxes (optional)
    for centroid in heatmap_data:
        cv2.circle(frame, centroid, 5, (0, 255, 0), -1)  # Draw dots on tracked positions

    cv2.imshow('Tracking', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Create a heatmap
heatmap, xedges, yedges = np.histogram2d(
    [p[0] for p in heatmap_data], [p[1] for p in heatmap_data], bins=(100, 100)
)

# Display the heatmap
plt.imshow(heatmap.T, cmap='jet', origin='lower', alpha=0.75)
plt.colorbar(label="Foot Traffic Density")
plt.title("Store Traffic Heatmap")
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.show()


: 